<a href="https://colab.research.google.com/github/Scaglione-Nick/ECGR4106/blob/main/HW5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

# Data Loading Function
def load_data_from_txt(file_path):
    english_sentences = []
    french_sentences = []

    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()

        # Join all lines into one string (to handle multiline tuples)
        content = ''.join(lines)

        # Regular expression to find all English-French pairs
        pattern = r'\("([^"]+)", "([^"]+)"\)'

        # Find all matching pairs (English, French)
        matches = re.findall(pattern, content)

        for match in matches:
            english_sentences.append(match[0])  # English sentence
            french_sentences.append(match[1])   # French sentence

    return english_sentences, french_sentences

english_sentences, french_sentences = load_data_from_txt('E2F_dataset.txt')

In [ ]:
# Tokenize the sentences
tokenizer_en = Tokenizer(oov_token='<OOV>')  # Handle out-of-vocabulary words in English
tokenizer_fr = Tokenizer(oov_token='<OOV>')  # Handle out-of-vocabulary words in French

tokenizer_en.fit_on_texts(english_sentences)
tokenizer_fr.fit_on_texts(french_sentences)

# Convert sentences to sequences of integers
encoder_input = tokenizer_en.texts_to_sequences(english_sentences)
decoder_input = tokenizer_fr.texts_to_sequences(french_sentences)

# Add <start> and <end> tokens to French sentences (for the decoder)
start_token = tokenizer_fr.word_index.get('<start>', len(tokenizer_fr.word_index) + 1)
end_token = tokenizer_fr.word_index.get('<end>', len(tokenizer_fr.word_index) + 2)
#Manually add <start> and <end> tokens to the tokenizer's word_index.
tokenizer_fr.word_index['<start>'] = start_token
tokenizer_fr.word_index['<end>'] = end_token

decoder_input = [[start_token] + seq + [end_token] for seq in decoder_input]
max_encoder_len = max([len(seq) for seq in encoder_input])
max_decoder_len = max([len(seq) for seq in decoder_input])

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    # Self-attention layer
    attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = layers.Dropout(dropout)(attention)
    attention = layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feed-forward network
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    output = layers.LayerNormalization(epsilon=1e-6)(attention + ff)

    return output

def transformer_decoder(inputs, encoder_output, head_size, num_heads, ff_dim, dropout=0.1):
    # Cross-attention layer
    attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, encoder_output)
    attention = layers.Dropout(dropout)(attention)
    attention = layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feed-forward network
    ff = layers.Dense(ff_dim, activation='relu')(attention)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    output = layers.LayerNormalization(epsilon=1e-6)(attention + ff)

    return output

def build_transformer(vocab_size_en, vocab_size_fr, max_len_encoder, max_len_decoder, head_size=128, num_heads=1, ff_dim=256, num_layers=1):
    # Encoder input
    encoder_input_layer = Input(shape=(max_len_encoder,))
    encoder_embedding = Embedding(input_dim=vocab_size_en, output_dim=head_size)(encoder_input_layer)  # Use vocab_size_en here

    encoder_output = encoder_embedding
    for _ in range(num_layers):
        encoder_output = transformer_encoder(encoder_output, head_size, num_heads, ff_dim)

    # Decoder input
    decoder_input_layer = Input(shape=(max_len_decoder,))
    decoder_embedding = Embedding(input_dim=vocab_size_fr, output_dim=head_size)(decoder_input_layer)  # Use vocab_size_fr here

    decoder_output = decoder_embedding
    for _ in range(num_layers):
        decoder_output = transformer_decoder(decoder_output, encoder_output, head_size, num_heads, ff_dim)

    # Final output layer
    output = layers.Dense(vocab_size_fr, activation='softmax')(decoder_output)  # Use vocab_size_fr here as well

    model = Model([encoder_input_layer, decoder_input_layer], output)
    return model

# Parameters
vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1

# Build the model with simplified parameters
model = build_transformer(vocab_size_en+20, vocab_size_fr+20, max_encoder_len, max_decoder_len, num_heads=2, num_layers=1, head_size=128, ff_dim=256)
# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 7)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_18 (Embedding)  │ (None, 7, 128)         │         34,304 │ input_layer_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_34   │ (None, 7, 128)         │        131,968 │ embedding_18[0][0],    │
│ (MultiHeadAttention)      │                        │                │ embedding_18[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_103 (Dropout)     │ (None, 7, 128)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_68 (Add)              │ (None, 7, 128)         │              0 │ embedding_18[0][0],    │
│                           │                        │                │ dropout_103[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_68    │ (None, 7, 128)         │            256 │ add_68[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_77 (Dense)          │ (None, 7, 256)         │         33,024 │ layer_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_104 (Dropout)     │ (None, 7, 256)         │              0 │ dense_77[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_78 (Dense)          │ (None, 7, 128)         │         32,896 │ dropout_104[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_19            │ (None, 11)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_69 (Add)              │ (None, 7, 128)         │              0 │ layer_normalization_6… │
│                           │                        │                │ dense_78[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_19 (Embedding)  │ (None, 11, 128)        │         38,272 │ input_layer_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_69    │ (None, 7, 128)         │            256 │ add_69[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_35   │ (None, 11, 128)        │        131,968 │ embedding_19[0][0],    │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_106 (Dropout)     │ (None, 11, 128)        │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_70 (Add)         

 Total params: 507,947 (1.94 MB)

 Trainable params: 507,947 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Pad the sequences to ensure that all sequences have the same length
encoder_input_padded = pad_sequences(encoder_input, maxlen=max_encoder_len, padding='post')
decoder_input_padded = pad_sequences(decoder_input, maxlen=max_decoder_len, padding='post')

# Prepare decoder output (next word prediction for the decoder)
decoder_output_data = [seq[1:] for seq in decoder_input]  # Create a list of target sequences

# Pad the decoder output data
decoder_output_padded = pad_sequences(decoder_output_data, maxlen=max_decoder_len, padding='post')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(encoder_input_padded, decoder_output_padded, test_size=0.2, random_state=42)
decoder_input_train, decoder_input_val = train_test_split(decoder_input_padded, test_size=0.2, random_state=42)

In [ ]:
# Train the model
history = model.fit(
    [X_train, decoder_input_train],
    y_train,
    epochs=10,               # Set the number of epochs
    batch_size=64,           # Adjust batch size as needed
    validation_data=([X_val, decoder_input_val], y_val)
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 981ms/step - accuracy: 0.1024 - loss: 5.4186 - val_accuracy: 0.4704 - val_loss: 3.6059
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.4740 - loss: 3.4927 - val_accuracy: 0.4704 - val_loss: 3.2630
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.4873 - loss: 3.1809 - val_accuracy: 0.4704 - val_loss: 3.0734
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.4761 - loss: 2.9203 - val_accuracy: 0.4704 - val_loss: 2.9103
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4960 - loss: 2.7158 - val_accuracy: 0.5296 - val_loss: 2.7479
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.5397 - loss: 2.4163 - val_accuracy: 0.5850 - val_loss: 2.6454
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.6291 - loss: 2.2059 - val_accuracy: 0.6206 - val_loss: 2.4964
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.6388 - loss: 1.9815 - val_accuracy: 0.6047 - val_loss:

In [ ]:
# List of configurations: (num_layers, num_heads)
configs = [
    (1, 2),  # 1 layer, 2 heads
    (1, 4),  # 1 layer, 4 heads
    (2, 2),  # 2 layers, 2 heads
    (2, 4),  # 2 layers, 4 heads
    (4, 2),  # 4 layers, 2 heads
    (4, 4)   # 4 layers, 4 heads
]

# Function to build and train the model for a given configuration
def train_transformer_with_config(num_layers, num_heads):
    print(f"Training model with {num_layers} layers and {num_heads} heads...")

    # Build the model with the given configuration
    model = build_transformer(vocab_size_en+2, vocab_size_fr+2, max_encoder_len, max_decoder_len, num_heads=num_heads, num_layers=num_layers, head_size=128, ff_dim=256)

    # Compile the model
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        [X_train, decoder_input_train],
        y_train,
        epochs=10,               # Set the number of epochs
        batch_size=64,           # Adjust batch size as needed
        validation_data=([X_val, decoder_input_val], y_val)
    )

    # Return the history object for analysis
    return history

# Loop through each configuration and train the model
history_results = {}
for num_layers, num_heads in configs:
    history_results[f"{num_layers}_layers_{num_heads}_heads"] = train_transformer_with_config(num_layers, num_heads)


Training model with 1 layers and 2 heads...
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 673ms/step - accuracy: 0.0880 - loss: 5.5517 - val_accuracy: 0.4704 - val_loss: 3.7029
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.4725 - loss: 3.4834 - val_accuracy: 0.4704 - val_loss: 3.3171
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.4878 - loss: 3.2023 - val_accuracy: 0.4704 - val_loss: 3.1552
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.4772 - loss: 2.9601 - val_accuracy: 0.4704 - val_loss: 2.9714
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4889 - loss: 2.8240 - val_accuracy: 0.4862 - val_loss: 2.7826
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5150 - loss: 2.5771 - val_accuracy: 0.5652 - val_loss: 2.6970
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.5822 - loss: 2.3397 - val_accuracy: 0.5692 - val_loss: 2.6099
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.6117 - los

In [ ]:
model = build_transformer(vocab_size_en+2, vocab_size_fr+2, max_encoder_len, max_decoder_len, num_heads=2, num_layers=1, head_size=128, ff_dim=256)

    # Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
history = model.fit(
    [X_train, decoder_input_train],
    y_train,
    epochs=50,               # Set the number of epochs
    batch_size=64,           # Adjust batch size as needed
    validation_data=([X_val, decoder_input_val], y_val)
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 657ms/step - accuracy: 0.0862 - loss: 5.4422 - val_accuracy: 0.4704 - val_loss: 3.3948
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.4759 - loss: 3.3465 - val_accuracy: 0.4704 - val_loss: 3.1474
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.4725 - loss: 3.0857 - val_accuracy: 0.4822 - val_loss: 2.9897
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.4997 - loss: 2.8380 - val_accuracy: 0.4980 - val_loss: 2.8187
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5207 - loss: 2.5621 - val_accuracy: 0.5613 - val_loss: 2.6642
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.5683 - loss: 2.3481 - val_accuracy: 0.5455 - val_loss: 2.6090
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.5939 - loss: 2.1971 - val_accuracy: 0.5968 - val_loss: 2.5087
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.6367 - loss: 1.9796 - val_accuracy: 0.5810 - val_loss:

In [ ]:
def decode_sequence(input_seq, tokenizer_en, tokenizer_fr, model, max_decoder_len):
    """
    Decode a sequence (input sequence) to get the predicted output (French translation).
    This function generates one word at a time and feeds it back into the model.
    """
    # Get the initial encoder state by predicting the encoder output
    encoder_input_seq = np.expand_dims(input_seq, axis=0)

    # Initialize the decoder input with the <start> token
    start_token = tokenizer_fr.word_index.get('<start>', tokenizer_fr.word_index.get('<OOV>'))
    decoder_input_seq = np.array([[start_token]])

    # Store the decoded sequence (French translation)
    decoded_sentence = []

    for _ in range(max_decoder_len):
        # Predict the next word (token)
        preds = model.predict([encoder_input_seq, decoder_input_seq])

        # Get the predicted word (token) with the highest probability
        predicted_token = np.argmax(preds[0, -1, :])

        # If we reach the <end> token, stop generating
        if predicted_token == tokenizer_fr.word_index['<end>']:
            break

        # Convert the predicted token to the word
        predicted_word = tokenizer_fr.index_word.get(predicted_token, "")
        decoded_sentence.append(predicted_word)

        # Update the decoder input sequence with the predicted word
        decoder_input_seq = np.concatenate([decoder_input_seq, np.array([[predicted_token]])], axis=-1)

    return ' '.join(decoded_sentence)

# Select a few test sentences from your validation set (for example, the first 5 sentences)
test_sentences = X_val[:5]
true_translations = french_sentences[:5]

# Decode each input sentence and compare it with the true translation
for i, sentence in enumerate(test_sentences):
    print(f"Input (English): {english_sentences[i]}")
    predicted_translation = decode_sequence(sentence, tokenizer_en, tokenizer_fr, model, max_decoder_len)
    print(f"Predicted Translation (French): {predicted_translation}")
    print(f"True Translation (French): {true_translations[i]}")
    print("-" * 50)

Input (English): I am cold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Translation (French): nous visitons le musée
True Translation (French): J'ai froid
--------------------------------------------------
Input (English): You are tired
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Translation (French): nous faisons des biscuits
True Translation (French): Tu es fatigué
--------------------------------------------------
Input (English): He is hungry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Translation (French): nous voyagent 